# 01 - Data Loading and Preprocessing
This notebook handles the initial data loading, cleaning, and text preprocessing steps for the REIC paper analysis.

## 1. Import Required Libraries

In [7]:
import pandas as pd
import json
import numpy as np
import re
from gensim.utils import simple_preprocess
import spacy
from nltk.corpus import stopwords
import nltk

# Download NLTK stopwords
nltk.download('stopwords')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/etorebraga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2. Load and Inspect Raw Data

In [8]:
# Define file path
input_path = '../data/raw/sentencas_georef_test.json'

# Load JSON data
with open(input_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# Convert to DataFrame
df = pd.DataFrame(data)

# Display basic info and first few rows
print(f"Total records: {len(df)}")
print("\nDataFrame Info:")
df.info()
print("\nFirst few rows:")
df.head()

Total records: 11

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   processo              11 non-null     object 
 1   Classe 1ª Instância   11 non-null     object 
 2   Assunto 1ª Instância  11 non-null     object 
 3   Comarca 1ª Instância  11 non-null     object 
 4   foro                  11 non-null     object 
 5   vara                  11 non-null     object 
 6   julgado               11 non-null     object 
 7   cod_doc               11 non-null     object 
 8   Instância             11 non-null     object 
 9   sentença              11 non-null     object 
 10  Comarca               11 non-null     object 
 11  Requerinte            11 non-null     object 
 12  Réu                   11 non-null     object 
 13  Assunto 2ª Instância  0 non-null      object 
 14  Classe 2ª Instância   0 non-null      obj

,processo,Classe 1ª Instância,Assunto 1ª Instância,Comarca 1ª Instância,foro,vara,julgado,cod_doc,Instância,sentença,Comarca,Requerinte,Réu,Assunto 2ª Instância,Classe 2ª Instância,cd_processo,data,dispositivo,cdacordao,Comarca 2ª Instância,orgao_julgador,ementa,Recurso 2º Grau,cd_doc,Assunto 2º Instância,latitude,longitude
0,00003463420238260619,Cumprimento de Sentença contra a Fazenda Pública,PROFISSIONAIS DE APOIO,Taquaritinga,Foro de Taquaritinga,Juizado Especial Cível e Criminal,"TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COMARCA de Taquaritinga Foro de Taquaritinga Juizado Especial Cível e Criminal Rua Visconde do Rio Branco, 71, Taquaritinga-SP - cep 15900-000 Horário de...",H70005JZY0000-619-PG5ARQA-45511889,1º Instância,procedente,taquaritinga,Pessoa Física,Estado,None,None,None,None,None,None,None,None,None,None,None,None,-21.425208,-48.537399
1,10019409220238260481,Mandado de Segurança Cível,Estabelecimentos de Ensino,Presidente Epitácio,Foro de Presidente Epitácio,1ª Vara,"TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COMARCA de Presidente Epitácio Foro de Presidente Epitácio 1ª Vara Av.Presidente Vargas 1-31, Presidente Epitacio - SP - cep 19470-000 Horário de Atendim...",DD0004XTS0000-481-PG5PP-84166446,1º Instância,procedente,presidente epitacio,Pessoa Física,Escola Pública,None,None,None,None,None,None,None,None,None,None,None,None,-21.765074,-52.111140
2,10039394320238260073,Procedimento do Juizado Especial Cível,Prestação de Serviços,Cerqueira César,Foro de Cerqueira César,Juizado Especial Cível e Criminal,"TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COMARCA de Cerqueira César Foro de Cerqueira César Juizado Especial Cível e Criminal Rua Olimpio Pavan nº 355, Cerqueira Cesar - SP - cep 18760-000 10039...",210004Q3A0000-136-PG5ARCT-114815115,1º Instância,procedente,cerqueira cesar,Pessoa Física,Pessoa Física,None,None,None,None,None,None,None,None,None,None,None,None,-23.035314,-49.165052
3,10071289220228260322,Mandado de Segurança Cível,Estabelecimentos de Ensino,Lins,Foro de Lins,2ª Vara Cível,"TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COMARCA de Lins - Foro de Lins - 2ª Vara Cível Rua Gil Pimentel Moura, 51, Centro CEP: 16400-920 - Lins - SP Telefone: (14) 3511-1525 - E-mail: Lins2cv@t...",8Y0003PG10000-322-PG5ARCT-114470170,1º Instância,parcial,lins,Pessoa Física,Pessoa Física,None,None,None,None,None,None,None,None,None,None,None,None,-21.649421,-49.682866
4,10001341720238260515,Procedimento Comum Cível,Matrícula - Ausência de Pré-Requisito,Rosana,Foro de Rosana,Vara Única,"TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COMARCA de Rosana Foro de Rosana Vara Única Rua Curimbatá, 788/802 QD 12 - D. Primavera, Primavera - SP - cep 19274-000 Horário de Atendimento ao Público...",EB00018UI0000-515-PG5SJCA-131051340,1º Instância,procedente,rosana,Pessoa Física,Estado,None,None,None,None,None,None,None,None,None,None,None,None,-22.488279,-52.836266


## 3. Text Preprocessing

In [9]:
# Define Portuguese stopwords
portuguese_stopwords = set(stopwords.words('portuguese'))

# Add custom stopwords
custom_stopwords = set([
    # Artigos
    'a', 'o', 'as', 'os', 'um', 'uma', 'uns', 'umas',

    # Preposições
    'ante', 'após', 'até', 'com', 'contra', 'de', 'desde', 'em', 'entre',
    'para', 'per', 'perante', 'por', 'sem', 'sob', 'sobre', 'trás',
    'durante', 'mediante', 'salvo', 'exceto', 'conforme', 'segundo',

    # Conjunções
    'e', 'mas', 'ou', 'que', 'se', 'porque', 'pois', 'porém',
    'todavia', 'contudo', 'entretanto', 'portanto', 'logo', 'assim',
    'então', 'por isso', 'nem', 'tanto', 'quanto', 'não só', 'mas também',
    'caso', 'embora', 'apesar', 'mesmo que', 'ainda que',

    # Pronomes
    'eu', 'tu', 'ele', 'ela', 'nós', 'vós', 'eles', 'elas',
    'me', 'mim', 'comigo', 'te', 'ti', 'contigo', 'se', 'si', 'consigo',
    'nos', 'conosco', 'vos', 'convosco', 'lhe', 'lhes', 'o', 'a', 'os', 'as',
    'meu', 'minha', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas',
    'seu', 'sua', 'seus', 'suas', 'nosso', 'nossa', 'nossos', 'nossas',
    'vosso', 'vossa', 'vossos', 'vossas', 'este', 'esta', 'estes', 'estas',
    'isto', 'esse', 'essa', 'esses', 'essas', 'isso', 'aquele', 'aquela',
    'aqueles', 'aquelas', 'aquilo', 'qual', 'quais', 'quem', 'cujo', 'cuja',
    'cujos', 'cujas', 'quanto', 'quanta', 'quantos', 'quantas',

    # Verbos Auxiliares (em várias conjugações)
    'sou', 'és', 'é', 'somos', 'sois', 'são', 'era', 'eras', 'éramos',
    'éreis', 'eram', 'fui', 'foste', 'foi', 'fomos', 'fostes', 'foram',
    'sido', 'estar', 'estou', 'estás', 'está', 'estamos', 'estais', 'estão',
    'estava', 'estavas', 'estávamos', 'estáveis', 'estavam', 'estive',
    'estiveste', 'esteve', 'estivemos', 'estivestes', 'estiveram', 'estado',
    'ter', 'tenho', 'tens', 'tem', 'temos', 'tendes', 'têm', 'tinha',
    'tinhas', 'tínhamos', 'tínheis', 'tinham', 'tive', 'tiveste', 'teve',
    'tivemos', 'tivestes', 'tiveram', 'tido', 'haver', 'hei', 'hás', 'há',
    'havemos', 'haveis', 'hão', 'havia', 'havias', 'havíamos', 'havíeis',
    'haviam', 'houve', 'houveste', 'houvemos', 'houvestes', 'houveram', 'havido',

    # Advérbios de Modo
    'assim', 'bem', 'mal', 'depressa', 'devagar', 'rapidamente', 'lentamente',
    'calmamente', 'propositadamente', 'inadvertidamente', 'fielmente',

    # Advérbios de Tempo
    'agora', 'hoje', 'amanhã', 'ontem', 'antes', 'depois', 'ainda', 'já',
    'sempre', 'nunca', 'jamais', 'atualmente', 'provisoriamente', 'entretanto',
    'primeiramente', 'ultimamente', 'anteriormente', 'posteriormente',

    # Advérbios de Lugar
    'aqui', 'ali', 'lá', 'acolá', 'cá', 'aí', 'além', 'aquém', 'dentro',
    'fora', 'acima', 'abaixo', 'adiante', 'atrás', 'junto', 'próximo',

    # Termos Jurídicos Comuns
    ## Termos Processuais
    'ação', 'processo', 'juiz', 'juíza', 'réu', 'ré', 'autor', 'autora',
    'advogado', 'advogada', 'procurador', 'procuradora', 'testemunha',
    'audiência', 'sentença', 'decisão', 'despacho', 'petição', 'recurso',
    'agravo', 'apelação', 'embargos', 'mandado', 'citação', 'intimação',
    'execução', 'liminar', 'tutela', 'protocolo', 'autos', 'vara', 'comarca',
    'fórum', 'tribunal', 'jurisdição', 'instância', 'intimação', 'intimações',
    'noticação', 'notificações', 'impugnação', 'impugnações', ''

    ## Termos Substantivos
    'direito', 'lei', 'justiça', 'crime', 'pena', 'multa', 'sanção',
    'obrigação', 'dever', 'contrato', 'acordo', 'norma', 'legislação',
    'jurisprudência', 'doutrina', 'constituição', 'decreto', 'portaria',
    'resolução', 'parecer', 'advocacia', 'magistratura', 'ministério',

    ## Termos Adjetivos
    'civil', 'penal', 'trabalhista', 'administrativo', 'constitucional',
    'processual', 'judicial', 'jurídico', 'legal', 'ilegal', 'legítimo',
    'ilegítimo', 'público', 'privado', 'competente', 'incompetente',

    ## Expressões Latinas Comuns
    'in', 'casu', 'ipso', 'facto', 'jure', 'data', 'venia', 'habeas',
    'corpus', 'mandamus', 'ex', 'vi', 'lato', 'sensu', 'stricto',

    ## Órgãos Judiciários
    'stf', 'stj', 'trf', 'trt', 'tjsp', 'tribunal', 'superior', 'supremo',
    'federal', 'justiça', 'vara', 'juizado', 'especial', 'turma', 'câmara',

    ## Outros Termos Jurídicos
    'acolho', 'defiro', 'indefiro', 'julgo', 'determino', 'cite-se',
    'intime-se', 'publique-se', 'certifico', 'concluso', 'visto',
    'considerando', 'posto', 'isto', 'diante', 'exposto', 'termos',
    'pedido', 'requer', 'requerente', 'requerido', 'impetrante', 'impetrado',
    'exequente', 'executado', 'foro',

    # Expressões de Conexão
    'face', 'ademais', 'outrossim', 'destarte', 'porquanto', 'consoante',
    'conforme', 'segundo', 'mediante', 'conseguinte', 'mister', 'faz-se',

    # Números e Medidas
    'primeiro', 'segunda', 'terceiro', 'quarto', 'quinta', 'artigo', 'art',
    'parágrafo', 'inciso', 'alínea', 'item', 'número', 'nr', 'nº',

    # Palavras Genéricas
    'ser', 'fazer', 'dizer', 'ver', 'dar', 'ir', 'vir', 'ficar', 'poder',
    'dever', 'parte', 'partes', 'cada', 'todo', 'toda', 'todos', 'todas',
    'outro', 'outra', 'outros', 'outras', 'mesmo', 'mesma', 'mesmos',
    'mesmas', 'muito', 'muita', 'muitos', 'muitas', 'pouco', 'pouca',
    'poucos', 'poucas', 'tão', 'quão', 'ora', 'seja', 'fosse', 'seria',
    'sido', 'fora',

    # Termos nulo e sobrantes
    'nan', 'que', 'pre',

])

# Combine all stopwords
all_stopwords = portuguese_stopwords.union(custom_stopwords)

def preprocess_text(text):
    """
    Preprocess text by:
    1. Converting to lowercase
    2. Removing special characters and numbers
    3. Tokenizing and removing stopwords
    4. Lemmatizing (if spaCy model is available)
    """
    if not isinstance(text, str):
        return []
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-záéíóúâêîôûãõç\s]', ' ', text)
    
    # Tokenize and remove stopwords
    tokens = [token for token in simple_preprocess(text) if token not in all_stopwords]
    
    # Lemmatization (if spaCy model is available)
    try:
        nlp = spacy.load('pt_core_news_sm')
        doc = nlp(' '.join(tokens))
        tokens = [token.lemma_ for token in doc]
    except:
        pass
    
    return tokens

## 4. Apply Preprocessing and Save Processed Data

In [10]:
# Apply preprocessing to the text column
print("Preprocessing text data...")
df['processed_text'] = df['julgado'].apply(preprocess_text)

# Save processed data
output_path = '../data/processed/data_processed_test.json'
df.to_json(output_path, orient='records', lines=True, force_ascii=False)
print(f"\nPreprocessing complete! Processed data saved to {output_path}")

# Display sample of processed text
print("\nSample of processed text:")
for i, tokens in enumerate(df['processed_text'].head(2)):
    print(f"\nDocument {i+1}:")
    print(' '.join(tokens[:50]) + '...' if len(tokens) > 50 else ' '.join(tokens))

Preprocessing text data...

Preprocessing complete! Processed data saved to ../data/processed/data_processed_test.json

Sample of processed text:

Document 1:
Paulo taquaritinga taquaritinga cível criminal rua visconder Rio branco taquaritinga sp cep horário atendimento Min Min laudar digital classe assunto cumprimento fazendo público profissional apoio Miguel augusto Ferreira Rodrigues prefeitura municipal taquaritinga dr Leopoldo vilela Andrade Silva Costa visto representar genitora ingressar presente incidente cumprimento visar restabelecer fornecimento professor auxiliar...

Document 2:
Paulo presidente epitácio presidente epitácio av presidente Vargas presidente epitacio sp cep horário atendimento Min Min laudar digital classe assunto segurança cível ensino éverton Lima barbosa dirigente regional ensino santo anastácio agente coator diretora geralda helenice augustar rocha gratuito conclusão janeiro fazer concluso José Roberto duran Garcia júnior assistente judiciário...


## 5. Create Corpus and Dictionary

In [11]:
from gensim import corpora

# Create dictionary
dictionary = corpora.Dictionary(df['processed_text'])

# Filter out extremes
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Create corpus
corpus = [dictionary.doc2bow(text) for text in df['processed_text']]

# Save dictionary and corpus
dictionary.save('../data/processed/dictionary_test.dict')

## Next Steps

The data has been preprocessed and is ready for exploratory data analysis. Proceed to the next notebook:
`02_exploratory_data_analysis.ipynb`